In [ ]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import skimage.io
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.patches import Polygon
from urllib.parse import urlparse
from cellpose import models, core, utils
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from skimage import exposure
import cv2
from skimage.transform import resize
from training_toolbox import resize_image

%matplotlib inline
Image.MAX_IMAGE_PIXELS = None
mpl.rcParams['figure.dpi'] = 200

# call logger_setup to have output of cellpose written
from cellpose.io import logger_setup
logger_setup();

# Prepare whole-slide fluorescent microscopy images

In [ ]:
# for segmentation visualization on more than 3 channel tiff, and to make sure the channel sequence is R G B Orange, actually use RGB format image as the reference image has better performance.
channel_sequence = input("Please input the channel sequence of the reference image, and use spaces to separate them (e.g. R G B Orange): ").split(" ")
red_index, green_index, blue_index, orange_index = -1, -1, -1, -1
for i in range(len(channel_sequence)):
    if channel_sequence[i].lower() == 'orange':
        orange_index = i
    elif channel_sequence[i].lower() == 'r':
        red_index = i
    elif channel_sequence[i].lower() == 'g':
        green_index = i
    elif channel_sequence[i].lower() == 'b':
        blue_index = i

In [ ]:
import tifffile

img = tifffile.imread("D:\cell_formal\OneDrive_2025-06-27\Oscar_images\BJ Ras\\type4-multi-stain (RGB)-3.tif")
# use tiffile to read the tif image, it can read one page with 3-channel RGB and 4 page with one
# channel in each page.
print("Original shape:", img.shape)
print("Original dtype:", img.dtype)
assert(img.dtype=='uint8')
if img.shape[0] < 10:
    img = np.moveaxis(img, 0, -1)  # (4, H, W) -> (H, W, 4)
    print("After shape:", img.shape)
red_channel = img[:,:,red_index:red_index+1]
green_channel = img[:,:,green_index:green_index+1]
blue_channel = img[:,:,blue_index:blue_index+1]
if orange_index != -1: # there is another orange channel apart from R G B.
    orange_channel = img[:,:,orange_index:orange_index+1]
    img = np.concatenate([red_channel, green_channel, blue_channel, orange_channel], axis=2)
    print("The channel sequence has been modified to R G B and Orange")
else:
    img = np.concatenate([red_channel, green_channel, blue_channel], axis=2)
    print("The channel sequence has been modified to R G B")
merged_img = Image.fromarray(img) # this image is reference image, only used to train the Cellpose, it should be an RGB COLOR tif image, not 4-channel tif image

In [ ]:
output_size = (4000, 1000)  # Adjust based on your CPU memory constraints. not too small, and the ratio between width and height should keep the same
downsize_image, original_size = resize_image(merged_img, output_size)

print(downsize_image.shape) # downsize_image is numpy, so the output shape is height x width
print(original_size) # original size is the size of Image, so the output shape is width x height

# Plot the downsized image
plt.imshow(downsize_image[:,:,:3])
plt.title('Downsize Image')
plt.axis('off')
plt.tight_layout()
plt.show()

# Pre-processing with CLAHE

In [ ]:
# Apply adaptive histogram equalization
equalized_image = exposure.equalize_adapthist(downsize_image[:,:,:3], clip_limit=0.02) # equalize_adapthist only accept an image with 3 channel，actually we need RGB COLOR tif file to do this, because the traditional tif file value is distributed in a narrow range, like [0,70], the contrast is low, so this equalize_adapthist cannot process well, but using RGB color in Fiji can normalize the pixel value to [0,255], the contrast is good, so this function works well

# Plot the original and preprocessed images
fig, axes = plt.subplots(1, 2, figsize=(15, 7))
axes[0].imshow(downsize_image[:,:,:3], cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(equalized_image, cmap='gray')
axes[1].set_title('Equalized Image')
axes[1].axis('off')

plt.show()

print(equalized_image.shape)

# Model training

In [ ]:
# Load the pre-trained Cellpose model
model = models.Cellpose(model_type='cyto3')

# Predict on the resized image
masks, flows, styles, diams = model.eval(equalized_image, diameter=None, channels=[2, 3])
print(masks.shape)

# Generate outlines from the segmentation mask
outlines = utils.outlines_list(masks)

# Plot the downsized image with outlines
plt.figure(figsize=(8, 8))
plt.imshow(equalized_image, cmap='gray')

for outline in outlines:
    polygon = Polygon(outline, fill=False, edgecolor='red', linewidth=1)
    plt.gca().add_patch(polygon)
plt.title('Resized Image with Outlines')
plt.axis('off')
plt.show()

# Mask upsizing

In [ ]:
pil_image = Image.fromarray(masks)
upsize_masks, original_size = resize_image(pil_image, (22656, 5594)) # based on the original size of the input image

print(np.array(merged_img).shape)
print(upsize_masks.shape)
print(original_size)

# Generate outlines from the segmentation mask
outlines = utils.outlines_list(upsize_masks)

# Plot the original image with outlines
plt.figure(figsize=(8, 8))
plt.imshow(img[:,:,:3], cmap='gray')
for outline in outlines:
    polygon = Polygon(outline, fill=False, edgecolor='red', linewidth=1)
    plt.gca().add_patch(polygon)
plt.title('Original Image with Outlines')
plt.axis('off')
plt.show()

# Extracted Individual Cells, use a 3-channel RGB Color tif to train the model, but segment on a 4-channel 8-bit tif image (or we can still use 3-channel RGB, based on the case)

In [ ]:
channel_sequence = input("Please input the channel sequence of the image you want to segment, and use spaces to separate them (e.g. R G B Orange): ").split(" ")
red_index, green_index, blue_index, orange_index = -1, -1, -1, -1
for i in range(len(channel_sequence)):
    if channel_sequence[i].lower() == 'orange':
        orange_index = i
    elif channel_sequence[i].lower() == 'r':
        red_index = i
    elif channel_sequence[i].lower() == 'g':
        green_index = i
    elif channel_sequence[i].lower() == 'b':
        blue_index = i

In [ ]:
import tifffile

img = tifffile.imread("D:\cell_formal\OneDrive_2025-06-27\Oscar_images\BJ Ras\\type4-multi-stain (RGB)-3.tif")
# use tiffile to read the tif image, it can read one page with 3-channel RGB and 4 page with one
# channel in each page.
print("Original shape:", img.shape)
print("Original dtype:", img.dtype)
assert(img.dtype=='uint8')
if img.shape[0] < 10:
    img = np.moveaxis(img, 0, -1)  # (4, H, W) -> (H, W, 4)
    print("After shape:", img.shape)
red_channel = img[:,:,red_index:red_index+1]
green_channel = img[:,:,green_index:green_index+1]
blue_channel = img[:,:,blue_index:blue_index+1]
if orange_index != -1: # there is another orange channel apart from R G B.
    orange_channel = img[:,:,orange_index:orange_index+1]
    img = np.concatenate([red_channel, green_channel, blue_channel, orange_channel], axis=2)
    print("The channel sequence has been modified to R G B and Orange")
else:
    img = np.concatenate([red_channel, green_channel, blue_channel], axis=2)
    print("The channel sequence has been modified to R G B")
# this img is used to segment

In [ ]:
# Create a directory to save individual cell images
output_dir = 'type4\\3'
os.makedirs(output_dir, exist_ok=True)

# Find unique cells in the mask
cell_ids = np.unique(upsize_masks)
cell_ids = cell_ids[cell_ids != 0]  # Remove background (ID 0)

for cell_id in cell_ids:
    # Create a mask for the current cell
    cell_mask = upsize_masks == cell_id

    # Adjust the mask to match the image's shape
    if orange_index != -1:
        cell_mask_rgb = np.stack([cell_mask] * 4, axis=-1)
    else:
        cell_mask_rgb = np.stack([cell_mask] * 3, axis=-1)

    # Extract the cell from the image using the mask
    cell_img = img * cell_mask_rgb # use the real image which aims to segment

    # Find the bounding box of the cell to crop the image
    coords = np.argwhere(cell_mask)
    y0, x0 = coords.min(axis=0)
    y1, x1 = coords.max(axis=0) + 1  # slices are exclusive at the top

    # Crop the cell image with bounding box
    cell_img_cropped = cell_img[y0:y1, x0:x1]

    # Save the cropped cell image
    cell_filename = os.path.join(output_dir, f'cell_{cell_id}.tif')
    tifffile.imwrite(cell_filename, cell_img_cropped)

print(f'Individual cell images are saved in the directory: {output_dir}')